In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stat
import csv

4x4 board, start nw, se reward 1
n w s e (0.9)

In [5]:
width = 4
height = 4
fail = 0.3

def reward (state):
    if state == 15:
        return 10
    else:
        return -1

def move (state, choice):
    new = state
    if state != 15:
        if choice == 0:
            if state >= width:
                new = state - width
        elif choice == 1:
            if state % width != 0:
                new = state - 1
        elif choice == 2:
            if state < width * (height - 1):
                new = state + width
        else:
            if state % width != width - 1:
                new = state + 1
    else:
        if choice == 0:
            new = 3
        elif choice == 1:
            new = 0
        elif choice == 2:
            new = 12
    return new

def action (state, choice):
    rng = np.random.random()
    if rng < (1 - fail):
        return move (state, choice)
    elif rng < (1 - fail) + (fail / 3):
        return move (state, (choice + 1) % 4)
    elif rng < 1 - (fail / 3):
        return move (state, (choice + 2) % 4)
    else:
        return move (state, (choice + 3) % 4)
    
def argQmax (Q_X):
    maxim = max(Q_X)
    ind = []
    for i in range(0, len(Q_X)):
        if maxim == Q_X[i]:
            ind.append(i)
    return ind[np.random.randint(0, len(ind))]

In [6]:
def QRL (X, A, epsilon, tol_Q, rho_Q, gamma, episodes, ep_len):
    
    Q = [np.zeros([len(X), len(A)])]

    for k in range(1, episodes + 1):
        Q.append(Q[k - 1].copy())
        state = X[0]
        for n in range(0, ep_len):
            if np.random.random() > epsilon:
                choic = argQmax(Q[k][state])
            else:
                choic = np.random.randint(0, len(A))
            newState = action(state, choic)
            r = reward(newState)
            # do we need to worry about epsilon-greedy policy for Q values
            Q[k][state][choic] += rho_Q * (r + gamma * Q[k][newState][argQmax(Q[k][newState])] - Q[k][state][choic])
            state = newState
        
        if np.absolute(np.subtract(Q[k], Q[k - 1])).sum() < tol_Q:
            print(k)
            break
    
    return Q[k]

np.random.seed(1)
print(QRL([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [0, 1, 2, 3], 0.3, 0.1, 0.2, 0.8, 1, 1000000))

[[-0.22709869 -0.05523775  0.39866121  0.74426563]
 [ 0.93224912  1.70068283  1.76736026  1.98320117]
 [ 2.99725332  1.64290365  3.82865897  3.81975997]
 [ 5.10953609  3.6961849   7.21002914  4.73485669]
 [-0.16586271  1.5723188   1.00421598  2.01315901]
 [ 2.12679013  1.99215643  3.51946577  3.66417848]
 [ 5.31511694  4.52520812  5.4255106   6.08810344]
 [ 5.03446045  5.93255477 13.26860107  8.05271944]
 [ 0.66343638  2.67750947  3.63242983  4.79921823]
 [ 2.77544207  5.20937569  7.04820315  5.1523831 ]
 [ 6.62956652  6.18646568 10.18910269  9.62753473]
 [10.99676274 11.89318898 22.56045816 14.25587607]
 [ 5.17164084  4.36940973  4.26537551  6.91442108]
 [ 6.00004836  8.14425818  8.72186245 11.76877933]
 [12.08080385 12.22577963 16.30042089 21.60359619]
 [12.41045043  2.69055191  8.66171934 14.23806904]]
